In [8]:
import logging
from pathlib import Path
from Pegasus.api import *
import glob
import os
import pandas as pd
logging.basicConfig(level = logging.DEBUG)

#Properties
props = Properties()
props["dagman.retry"] = "100"
props["pegasus.transfer.arguments"] = "-m 1"
props.write()


# import os
# os.environ['KAGGLE_USERNAME'] = "vedula"
# os.environ['KAGGLE_KEY'] = "482a5c14ced45f63f3698eacb8fa0c62"

# import kaggle
# kaggle.api.dataset_download_files('nikhilpandey360/chest-xray-masks-and-labels/download', path='.', unzip=True)


preprocess = Transformation(
                "preprocess",
                site="local",
                pfn="/home/scitech/shared-data/mask_wf/preprocess.py",
                is_stageable=True
            )

data_split = Transformation(
                "data_split",
                site="local",
                pfn="/home/scitech/shared-data/mask_wf/data_split.py",
                is_stageable=True
            )

train_model = Transformation( 
                "train_model",
                site="local",
                pfn="/home/scitech/shared-data/mask_wf/train_model.py",
                is_stageable=True
            )


tc = TransformationCatalog().add_transformations(preprocess, data_split, train_model).write()

file_list = []
output_list = []

rc = ReplicaCatalog()

for file in glob.glob("./train_images/*.png"):
    f = file.replace("./train_images/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_images/").resolve() / f)
    
for file in glob.glob("./train_masks/*.png"):
    f = file.replace("./train_masks/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_masks/").resolve() / f)
    
for file in glob.glob("./test/*.png"):
    f = file.replace("./test/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./test/").resolve() / f)
    

    
checkpoint_file = "study_checkpoint.pkl"
if not os.path.isfile(checkpoint_file):
    df = pd.DataFrame(list())
    df.to_pickle(checkpoint_file)

rc.add_replica("local", checkpoint_file, Path(".").resolve() / checkpoint_file)
    
rc.write()

for filename in glob.glob("./train_images/*.png"):
    f = filename.replace("./train_images/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))


for filename in glob.glob("./train_masks/*.png"):
    f = filename.replace("./train_masks/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))

for filename in glob.glob("./test/*.png"):
    f = filename.replace("./test/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))

        
wf = Workflow("preprocess")
    
job_preprocess = Job(preprocess)\
                    .add_inputs(*file_list)\
                    .add_outputs(*output_list)

fb = File("data_split.pkl")

job_data_split = Job(data_split)\
                    .add_inputs(*output_list)\
                    .add_outputs(fb)

model = File("model.h5")


job_train = Job(train_model)\
                    .add_checkpoint(File(checkpoint_file), stage_out=True)\
                    .add_inputs(*output_list, fb)\
                    .add_profiles(Namespace.PEGASUS, key="maxwalltime", value=1)\
                    .add_outputs(model)


wf.add_jobs(job_preprocess, job_data_split, job_train)

try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)  


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.10.16 10:38:28.636 UTC:
2020.10.16 10:38:28.641 UTC:   -----------------------------------------------------------------------
2020.10.16 10:38:28.647 UTC:   File for submitting this DAG to HTCondor           : preprocess-0.dag.condor.sub
2020.10.16 10:38:28.653 UTC:   Log of DAGMan debugging messages                 : preprocess-0.dag.dagman.out
2020.10.16 10:38:28.658 UTC:   Log of HTCond

[###################################---------------]  70.0% ..Failure (Completed: 14, Queued: 0, Running: 1, Failed: 0)



####################
# pegasus-analyzer #
####################
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

Submit Directory   : /home/scitech/shared-data/mask_wf/scitech/pegasus/preprocess/run0047
Total jobs         :     20 (100.00%)
# jobs succeeded   :     14 (70.00%)
# jobs failed      :      0 (0.00%)
# jobs held        :      1 (5.00%)
# jobs unsubmitted :      6 (30.00%)

*******************************Held jobs' details*******************************

=============================train_model_ID0000003==============================

submit file            : train_model_ID0000003.sub
last_job_instance_id   : 13
reason                 :  Error from slot1@ebc750aada81: STARTER at 127.0.0.1 failed to send file(s) to <127.0.0.1:9618>: error reading from /var/lib/condor/execute/dir_48416/model.h5: (errno 2) No such file or directory; SHADOW failed to receive file(s) from <127.0.0.1:40683


Please run pegasus monitord in replay mode.

